In [ ]:
import networkx as nx
from tqdm.notebook import tqdm
import os
import statistics
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append( '.' )
import importlib

import measures
importlib.reload(measures)

In [4]:
def get_measures(start_path):

    measures_total = defaultdict(int)

    for file in tqdm(os.listdir(start_path + 'fake')):
        get_compare_measurements(file, measures_total, start_path)

    measures_percentage = defaultdict(int)

    for key in list(measures_total.keys()):
        measures_percentage[key] = round(measures_total[key]/len(os.listdir(start_path + 'fake')), 2)

    return measures_total, measures_percentage

def get_compare_measurements(file, measures_total, start_path):

    fake_file_path = start_path + 'fake' + '/' + file
    true_file_path = start_path + 'true' + '/' + file

    G_fake = measures.generate_graph_udpipe(fake_file_path)
    G_true = measures.generate_graph_udpipe(true_file_path)

    measures_fake = measures.get_measures_from_graph(G_fake)
    measures_true = measures.get_measures_from_graph(G_true)

    for key, value in list(measures_fake.items()):
        if  isinstance(value, dict):
            measures_fake[key] = max(value.values())

    for key, value in list(measures_true.items()):
        if  isinstance(value, dict):
            measures_true[key] = max(value.values())

    for key in measures_fake.keys():
        if measures_true[key] > measures_fake[key]:
            measures_total[key] += 1

def return_list_in_order(measures):
    measures_list = []

    measures_list.append(measures['betweenness'])
    measures_list.append(measures['closeness'])
    measures_list.append(measures['eigenvector'])
    measures_list.append(measures['katz'])
    measures_list.append(measures['pagerank'])
    measures_list.append(measures['hubs'])
    measures_list.append(measures['authorities'])

    measures_list.append(measures['density'])
    return measures_list

def return_keys_in_order():
    measures_list = []

    measures_list.append('betweenness')
    measures_list.append('closeness')
    measures_list.append('eigenvector')
    measures_list.append('katz')
    measures_list.append('pagerank')
    measures_list.append('hubs')
    measures_list.append('authorities')

    measures_list.append('density')
    return measures_list


In [ ]:
measures_without_stopwords, measures_percentage_without_stopwords = get_measures('data/udpipe_tuples/')
measures_with_stopwords, measures_percentage_with_stopwords = get_measures('data/udpipe_tuples_with_stopwords/')

In [6]:
cols = pd.MultiIndex.from_product([["with stopwords","without stopwords"], ['true>fake', '%']])

df_metrics = pd.DataFrame(np.array([return_list_in_order(measures_with_stopwords), return_list_in_order(measures_percentage_with_stopwords),
                                    return_list_in_order(measures_without_stopwords), return_list_in_order(measures_percentage_without_stopwords)]).T,
                            columns=cols, 
                            index=return_keys_in_order())

In [7]:
df_metrics = df_metrics.reset_index(col_level=1).rename(columns={'index':'Measure'})
df_metrics.to_csv("../results/compare_metrics_syntatic_greater_max.csv", index=False)

In [8]:
df_metrics = pd.read_csv("../results/compare_metrics_syntatic_greater_mean.csv", header=[0,1], index_col=0)
df_metrics.style.highlight_min(axis = 0, props='color:lightgreen;').format(precision=2).set_caption("Mean node values")

In [10]:
df_metrics = pd.read_csv("../results/compare_metrics_syntatic_greater_max.csv", header=[0,1], index_col=0)
df_metrics.style.highlight_min(axis = 0, props='color:lightgreen;').format(precision=2).set_caption("Max node values")